In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
import biolqm
import numpy as np
import boolean
import colomoto.minibn as minibn

### Building STG table

##### First, loading the model as a minibn

In [12]:
bnet_model = biolqm.load("model_files/krasmodel15vars.bnet", "bnet")

In [14]:
model = biolqm.to_minibn(bnet_model)

In [16]:
nodes = list(model.keys())
nodes

['ATM',
 'ATR',
 'CDC25B',
 'CHEK1',
 'DSB',
 'FAHRR',
 'FANCD2I',
 'FAcore',
 'HRR2',
 'KRAS',
 'MAPKAPK2',
 'NHEJ',
 'cc',
 'cell_death',
 'g2m_trans']

In [18]:
n=len(nodes)

##### Then, building the list of binary states

In [54]:
x = np.array([range(pow(2, n))])
x

array([[    0,     1,     2, ..., 32765, 32766, 32767]])

In [152]:
y = np.array([np.power([2.0]*n, np.array(list(range(0, -n, -1))))])
y

array([[1.00000000e+00, 5.00000000e-01, 2.50000000e-01, 1.25000000e-01,
        6.25000000e-02, 3.12500000e-02, 1.56250000e-02, 7.81250000e-03,
        3.90625000e-03, 1.95312500e-03, 9.76562500e-04, 4.88281250e-04,
        2.44140625e-04, 1.22070312e-04, 6.10351562e-05]])

In [153]:
list_binary_states = np.remainder(np.floor(np.multiply(x.transpose(), y)), 2).astype(bool)

In [154]:
np.shape(list_binary_states)

(32768, 15)

##### Then, building the update matrix

In [155]:
def gen_array(formula, list_binary_states, nodes):

    if isinstance(formula, boolean.boolean.Symbol):
        return list_binary_states[:, nodes.index(str(formula))]
    
    elif isinstance(formula, boolean.boolean.NOT) or isinstance(formula, minibn.NOT):
        return numpy.logical_not(
            gen_array(formula.args[0], list_binary_states, nodes)
        )
    
    elif isinstance(formula, boolean.boolean.OR):
        ret = gen_array(formula.args[0], list_binary_states, nodes)
        for i in range(1, len(formula.args)):
            ret = numpy.logical_or(ret, 
                gen_array(formula.args[i], list_binary_states, nodes)
            )
        return ret
    
    elif isinstance(formula, boolean.boolean.AND):
        ret = gen_array(formula.args[0], list_binary_states, nodes)
        for i in range(1, len(formula.args)):
            ret = numpy.logical_and(ret, 
                gen_array(formula.args[i], list_binary_states, nodes)
            )
        return ret
    
    else:
        print("Unknown boolean operator : %s" % type(formula))

In [156]:
update_matrix = np.array(
    [
        gen_array(model[node], list_binary_states, nodes) 
        for node in nodes
    ]
)

In [157]:
np.shape(update_matrix)

(15, 32768)

It should be using scipy's sparse matricies, but for now it's ok

##### Build up/down trans source/target

In [162]:
# function inds=fcn_state_inds(yes_no,n_series_exp,n_isl_exp)

# n_series_exp=n_series_exp-1;
# n_isl_exp=n_isl_exp-1;
# yes_no=yes_no-1;

# inds=sum([
#     1:2^n_series_exp; reshape(
#         repmat(
#             (1:2^(n_series_exp-n_isl_exp))
#             +yes_no,2^n_isl_exp,1
#         ),
#         1,2^n_series_exp
#     )*2^n_isl_exp
# ]);

In [175]:
def fcn_states_inds(yes_no, n_series_exp, n_isl_exp):
#     n_series_exp = n_series_exp - 1
#     n_isl_exp = n_isl_exp - 1
#     yes_no = yes_no - 1

    return np.array([(np.array(range(pow(2, (n_series_exp-n_isl_exp))))+yes_no)]*pow(2, n_isl_exp))

In [177]:
np.shape(fcn_states_inds(0, n, 0))

(1, 32768)